In [10]:
from __future__ import print_function, division


# Getting at Data

Both SunPy and Astropy have utilities for downloading data for your delectation. They are simple and easy to use, however increasing levels of computing will allow a great deal of personalisation and selection. Let us begin with SunPy

## Aquiring Data with SunPy

### VS0
This is the main interface which SunPy to search for and find solar data. VSO stands for the Virtual Solar Observatory, a service which presents a homogenous interface to heterogenous data set.

So what do we need?

In [11]:
from sunpy.net import vso
client = vso.VSOClient()

This is your client object. This is effectively the intermediary between yourself and the treasure chest of solar data available. You query VSO, then VSO querys all data providers which fit the limiations you imposed during your search command. The VSO client also handles the particulars of dowloading the data onto your machiene. 

## Making a query

Lets kick off with an example, lets ask the veteran of solar imaging, SoHO for some EIS data, between the dates of between January 1st and 2nd, 2001,

In [12]:
qr = client.query(vso.attrs.Time('2001/1/1', '2001/1/2'), 
                  vso.attrs.Instrument('eit'))

`qr` is a Python list of response onjects, each one a record that the VSO has found.

In [13]:
print(len(qr))
print(qr)

122
   Start Time [1]       End Time [1]    Source Instrument   Type  
------------------- ------------------- ------ ---------- --------
2001-01-01 01:00:14 2001-01-01 01:00:21   SOHO        EIT FULLDISK
2001-01-01 07:00:14 2001-01-01 07:00:21   SOHO        EIT FULLDISK
2001-01-01 13:00:15 2001-01-01 13:00:22   SOHO        EIT FULLDISK
2001-01-01 19:00:14 2001-01-01 19:00:21   SOHO        EIT FULLDISK
2001-01-01 00:00:42 2001-01-01 00:00:54   SOHO        EIT FULLDISK
2001-01-01 00:12:10 2001-01-01 00:12:23   SOHO        EIT FULLDISK
2001-01-01 00:24:10 2001-01-01 00:24:22   SOHO        EIT FULLDISK
2001-01-01 00:36:10 2001-01-01 00:36:22   SOHO        EIT FULLDISK
2001-01-01 00:48:10 2001-01-01 00:48:22   SOHO        EIT FULLDISK
2001-01-01 01:13:45 2001-01-01 01:13:57   SOHO        EIT FULLDISK
                ...                 ...    ...        ...      ...
2001-01-01 23:24:10 2001-01-01 23:24:22   SOHO        EIT FULLDISK
2001-01-01 23:36:10 2001-01-01 23:36:22   SOHO        EIT 

### Break it down

So we can pass many attributes to the VSO, in this case we started with time

<pre><code> vso.attrs.Time('2001/1/1','2001/1/2')</code></pre>

Start and end times for the query as strings, any date/time function can be understood by SunPy's parse_time function e.g. the datetime onjects we will look at later. Next we give it the instrument we want:

<pre><code> vso.attrs.Instrument('eit') </code></pre>

You don't have to pass it an instrument, the client will find all available missions in the parameter you've defined if you like. Next, wavelength:

<pre><code> vso.attrs.Wave(14.2*u.nm, 12.3*u.nm)</code></pre> 

We pass it a min and max wavelength. This has to be an astropy units quantity (in SI for the love of coffee). If you don't you will get an error.
 
For a full list of attributes that vso can take use:



So we can use multiple instument queries and define get smaller sample times but narrowing down the query:

In [14]:
qr = client.query(vso.attrs.Time('2001/1/1T12:00:00', '2001/1/2T13:00:00'), vso.attrs.Instrument('eit') | vso.attrs.Instrument('mdi'))
len(qr)

3361

### HEK

The Heliophysics Event Knowledgebase (HEK) is a repository of feature and event information about the Sun. Entries are generated both by automated algorithms and human observers.

We need to set up HEK in a similar way to VSO

In [15]:
from sunpy.net import hek
hek_client = hek.HEKClient()

Creating a very similar client as we saw with VSO above.

Given that HEK is a database of solar events of interest, the query has different requirements to VSO. It needs start and end times, and an event type. Again time objects can be defined as datetime objects or correctly formatted strings.

Event types are specified as uppercase two letter strings found on [the HEK website](http://www.lmsal.com/hek/VOEvent_Spec.html)

In [16]:
tstart = '2011/08/09 07:23:56'
tend = '2011/08/09 12:40:29'
event_type = 'FL'
result = hek_client.query(hek.attrs.Time(tstart,tend),
                          hek.attrs.EventType(event_type))



Notice that the HEK query is extremely similar to the VSO query style, with our attributes defined accordingly.

Instead of returning a list, HEK returns a list of dictionary objects. Each entry in the dictionary sis a pair of key-value pairs that exactly correspond to the parameters. We can return the key words using:

In [17]:
result[0].keys()

[u'concept',
 u'frm_versionnumber',
 u'gs_movieurl',
 u'hrc_coord',
 u'fl_peakflux',
 u'hpc_bbox',
 u'area_atdiskcenter',
 u'event_mapurl',
 u'event_c1error',
 u'obs_dataprepurl',
 u'hgc_coord',
 u'frm_identifier',
 u'intensmean',
 u'boundbox_c2ur',
 u'event_coordunit',
 u'gs_thumburl',
 u'fl_peakfluxunit',
 u'obs_meanwavel',
 u'bound_ccnsteps',
 u'skel_curvature',
 u'hgs_coord',
 u'bound_chaincode',
 u'frm_daterun',
 u'fl_peakem',
 u'intensmedian',
 u'fl_efoldtimeunit',
 u'bound_ccstartc1',
 u'frm_paramset',
 u'bound_ccstartc2',
 u'event_coord2',
 u'event_coord3',
 u'event_coord1',
 u'event_importance',
 u'kb_archivdate',
 u'event_title',
 u'fl_peaktempunit',
 u'hgc_bbox',
 u'skel_chaincode',
 u'intenstotal',
 u'hrc_a',
 u'area_atdiskcenteruncert',
 u'event_probability',
 u'hrc_boundcc',
 u'event_description',
 u'eventtype',
 u'obs_channelid',
 u'ar_mcintoshcls',
 u'frm_institute',
 u'frm_contact',
 u'ar_noaaclass',
 u'boundbox_c1ur',
 u'hgs_boundcc',
 u'boundbox_c2ll',
 u'area_unit',

Remember, the HEK query we made returns all the flares in the time-range stored in the HEK, regardless of the feature recognition method. The HEK parameter which stores the the feature recognition method is called “frm_name”. Using list comprehensions (which are very cool), it is easy to get a list of the feature recognition methods used to find each of the flares in the result object, for example:

In [18]:
for elem in result: 
    print(elem["frm_name"])

asainz
asainz
asainz
asainz
asainz
asainz
asainz
SWPC
SSW Latest Events
Flare Detective - Trigger Module
Flare Detective - Trigger Module
SWPC
SSW Latest Events
Flare Detective - Trigger Module
Flare Detective - Trigger Module
Flare Detective - Trigger Module
Flare Detective - Trigger Module
Flare Detective - Trigger Module
Flare Detective - Trigger Module


This way we can avoid troublesome doubling up of results. We can do the same `help(hek.attrs)` command as VSO to fins out further options. 

## Aquiring data with AstroQuery

Astroquery supports a plethora of [services](https://astroquery.readthedocs.org/en/latest/#using-astroquery), all of which follow roughly the same API (application program interface). In its simplest for the API involves queries based on coordinates or object names e.g. using SIMBAD:

In [19]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object("m1")
result_table.pprint(show_unit=True)

MAIN_ID      RA         DEC     ... COO_QUAL COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"     "d:m:s"   ...                                            
------- ----------- ----------- ... -------- -------------- -------------------
  M   1 05 34 31.94 +22 00 52.2 ...        C              R 2011A&A...533A..10L


In this case the query is looking at a specific set of coordinates

In [20]:
from astropy import coordinates
import astropy.units as u
c = coordinates.SkyCoord("05h35m17.3s -05d23m28s", frame='icrs')
r = 5 * u.arcminute
result = Simbad.query_region(c, radius=r)
result.pprint(show_unit=True, max_width=80, max_lines=5)

        MAIN_ID              RA      ... COO_WAVELENGTH     COO_BIBCODE    
                          "h:m:s"    ...                                   
----------------------- ------------ ... -------------- -------------------
        NAME ORI REGION  05 35 17.30 ...              O                    
                    ...          ... ...            ...                 ...
2MASS J05353130-0519533  05 35 31.31 ...              I 2003yCat.2246....0C
           V* V2114 Ori 05 35 01.671 ...              I 2003yCat.2246....0C
Length = 2977 rows


These methods can be expanded to all the following modules


*    SIMBAD Queries (astroquery.simbad)
*    IRSA Dust Extinction Service Queries (astroquery.irsa_dust)
*    NED Queries (astroquery.ned)
*    Splatalogue Queries (astroquery.splatalogue)
*    IRSA Image Server program interface (IBE) Queries (astroquery.ibe)
*    IRSA Queries (astroquery.irsa)
*    UKIDSS Queries (astroquery.ukidss)
*    MAGPIS Queries (astroquery.magpis)
*    NRAO Queries (astroquery.nrao)
*    Besancon Queries (astroquery.besancon)
*    NIST Queries (astroquery.nist)
*    NVAS Queries (astroquery.nvas)
*    GAMA Queries (astroquery.gama)
*    ESO Queries (astroquery.eso)
*    Atomic Line List (astroquery.atomic)
*    ALMA Queries (astroquery.alma)
*    Skyview Queries (astroquery.skyview)
*    NASA ADS Queries (astroquery.nasa_ads)
*    HEASARC Queries (astroquery.heasarc)

